<a href="https://colab.research.google.com/github/aicreativeexplorer/YT-Automation/blob/main/YT_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# CONFIG — edit only if you moved files
DRIVE_TOKEN_PATH = "/content/drive/MyDrive/ai_shorts_pipeline/hf_token.txt"
SVD_VERSION = "svd-xt-1-1"   # use "svd" if you want the lighter open model
UPLOADED_VIDEO = "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4"
OUTPUT_DRIVE_FOLDER = "/content/drive/MyDrive/ai_shorts_pipeline/stitched"
CHECKPOINT_DIR = "/content/checkpoints"
print("CONFIG loaded")

CONFIG loaded


In [3]:
from google.colab import drive
from huggingface_hub import login
import os

print('Mounting Drive...')
drive.mount('/content/drive', force_remount=True)

if os.path.exists(DRIVE_TOKEN_PATH):
    with open(DRIVE_TOKEN_PATH, 'r') as f:
        hf_token = f.read().strip()
    login(token=hf_token)
    print('🔥 HuggingFace logged in from Drive token')
else:
    print('⚠️ hf_token.txt not found at', DRIVE_TOKEN_PATH)
    print('If you want, paste token interactively now:')
    from huggingface_hub import login as hf_login
    hf_login()


Mounting Drive...
Mounted at /content/drive
🔥 HuggingFace logged in from Drive token


In [5]:
print("Installing key packages (fast)..")
!pip install -q kornia open_clip_torch timm transformers accelerate safetensors pytorch-lightning fire gradio==3.34.0 omegaconf einops imageio-ffmpeg imwatermark
print("Install cell done")


Installing key packages (fast)..
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
Install cell done


In [6]:
# run this in a code cell (keep the !)
# idempotent: will skip clone if already present
!if [ ! -d /content/generative-models ]; then git clone https://github.com/Stability-AI/generative-models.git /content/generative-models; else echo "generative-models exists"; fi
!pip install -q -e /content/generative-models || true
print("generative-models clone+install step finished")


Cloning into '/content/generative-models'...
remote: Enumerating objects: 1140, done.
remote: Counting objects: 100% (562/562), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1140 (delta 438), reused 390 (delta 385), pack-reused 578 (from 3)
Receiving objects: 100% (1140/1140), 86.66 MiB | 13.60 MiB/s, done.
Resolving deltas: 100% (604/604), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for sgm (pyproject.toml) ... done
generative-models clone+install step finished


In [7]:
import sys, os
if "/content/generative-models" not in sys.path:
    sys.path.insert(0, "/content/generative-models")
print("sys.path updated")

try:
    import sgm
    print("✅ sgm import OK:", sgm.__file__)
except Exception as e:
    print("❌ sgm import failed:", e)
    raise


sys.path updated


✅ sgm import OK: /content/generative-models/sgm/__init__.py


In [8]:
from huggingface_hub import hf_hub_download
import os
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
TYPE2PATH = {
    "svd": ["stabilityai/stable-video-diffusion-img2vid","svd.safetensors"],
    "svd-xt": ["stabilityai/stable-video-diffusion-img2vid-xt","svd_xt.safetensors"],
    "svd-xt-1-1": ["stabilityai/stable-video-diffusion-img2vid-xt-1-1","svd_xt_1_1.safetensors"],
}
repo_id,fname = TYPE2PATH.get(SVD_VERSION)
ckpt_path = os.path.join(CHECKPOINT_DIR, fname)
if os.path.exists(ckpt_path):
    print("✔ Checkpoint exists:", ckpt_path)
else:
    print("⬇ Downloading checkpoint (may be big)...")
    hf_hub_download(repo_id=repo_id, filename=fname, local_dir=CHECKPOINT_DIR)
    print("✔ Downloaded:", ckpt_path)


⬇ Downloading checkpoint (may be big)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


svd_xt_1_1.safetensors:   0%|          | 0.00/4.78G [00:00<?, ?B/s]

✔ Downloaded: /content/checkpoints/svd_xt_1_1.safetensors


In [9]:
from omegaconf import OmegaConf
import torch
from sgm.util import instantiate_from_config

def load_model_local(config_path, ckpt_path, device='cuda', num_frames=14, num_steps=25):
    config = OmegaConf.load(config_path)
    config.model.params.ckpt_path = ckpt_path
    config.model.params.sampler_config.params.num_steps = num_steps
    config.model.params.sampler_config.params.guider_config.params.num_frames = num_frames
    with torch.device(device):
        model = instantiate_from_config(config.model).to(device).eval().requires_grad_(False)
    return model

if "xt" in SVD_VERSION:
    model_config = "/content/generative-models/scripts/sampling/configs/svd_xt.yaml"
    num_frames = 25; num_steps = 30
else:
    model_config = "/content/generative-models/scripts/sampling/configs/svd.yaml"
    num_frames = 14; num_steps = 25

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Loading model on", device)
model = load_model_local(model_config, ckpt_path, device=device, num_frames=num_frames, num_steps=num_steps)
print("🔥 Model loaded")


Loading model on cpu
VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing


...

VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing


VideoTransformerBlock is using checkpointing
VideoTransformerBlock is using checkpointing


open_clip_model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Initialized embedder #0: FrozenOpenCLIPImagePredictionEmbedder with 683800065 params. Trainable: False
Initialized embedder #1: ConcatTimestepEmbedderND with 0 params. Trainable: False
Initialized embedder #2: ConcatTimestepEmbedderND with 0 params. Trainable: False
Initialized embedder #3: VideoPredictionEmbedderWithEncoder with 83653863 params. Trainable: False
Initialized embedder #4: ConcatTimestepEmbedderND with 0 params. Trainable: False
Restored from /content/checkpoints/svd_xt_1_1.safetensors with 0 missing and 0 unexpected keys
🔥 Model loaded


In [10]:
# Extract a frame at 1s from your uploaded video
!ffmpeg -ss 00:00:01 -i "/mnt/data/Kling AI- Next-Gen AI Video & AI Image Generator.mp4" -frames:v 1 /content/frame_for_svd.png -y
print("Frame saved to /content/frame_for_svd.png")


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [14]:
# === Define infer() wrapper that calls the notebook's sample() if available ===
import random, os, traceback

def infer(input_path: str,
          resize_image: bool = True,
          n_frames: int = 14,
          n_steps: int = 25,
          seed: str = "random",
          decoding_t: int = 1,
          fps_id: int = 6,
          motion_bucket_id: int = 127,
          cond_aug: float = 0.02,
          skip_filter: bool = False) -> str:
    """
    Wrapper to call the notebook's sample(...) function and return the created video path.
    Requires the original notebook's sample(...) implementation to be present in the session.
    """
    if seed == "random":
        seed = random.randint(0, 2**31-1)
    try:
        # Try to call the existing sample() function used by the original notebook
        out_paths = sample(
            input_path=input_path,
            resize_image=resize_image,
            num_frames=int(n_frames),
            num_steps=int(n_steps),
            fps_id=int(fps_id),
            motion_bucket_id=int(motion_bucket_id),
            cond_aug=float(cond_aug),
            seed=int(seed),
            decoding_t=int(decoding_t),
            device=device,        # depends on the notebook Load Model cell setting
            skip_filter=skip_filter
        )
        # sample() usually returns a list of output paths
        if isinstance(out_paths, (list, tuple)) and len(out_paths)>0:
            return out_paths[0]
        elif isinstance(out_paths, str):
            return out_paths
        else:
            raise RuntimeError(f"sample() returned unexpected value: {out_paths}")
    except NameError as e:
        # sample() not defined — helpful guidance
        raise NameError(
            "sample() or model sampling functions are not defined in this session. "
            "Please run the notebook cells named 'Sampling function' or 'Do the Run!' that define sample()/infer().\n"
            "If you want, paste the original sample() implementation into a cell and re-run this cell."
        ) from e
    except Exception as e:
        # bubble up other errors with traceback for debugging
        traceback.print_exc()
        raise


In [16]:
# Try calling infer() if your notebook defines it. If NameError, run the Sampling / Do the Run! cells from the original notebook.
try:
    out_path = infer("/content/frame_for_svd.png", True, num_frames, num_steps, "random", 1, 6, 127, 0.02, skip_filter=True)
    print("Generated video at:", out_path)
    # show it
    from IPython.display import Video, display
    display(Video(out_path, embed=True, width=360))
except NameError:
    print("infer() is not defined in this session. Scroll up and run the original notebook's 'Sampling function' / 'Do the Run!' cells, or paste the sample()/infer() code into a cell.")
except Exception as e:
    print("infer() failed:", e)
    raise


infer() is not defined in this session. Scroll up and run the original notebook's 'Sampling function' / 'Do the Run!' cells, or paste the sample()/infer() code into a cell.


In [12]:
import shutil, os
os.makedirs(OUTPUT_DRIVE_FOLDER, exist_ok=True)
if 'out_path' in globals() and out_path:
    dst = os.path.join(OUTPUT_DRIVE_FOLDER, "demo_output.mp4")
    shutil.copy(out_path, dst)
    print("Saved output to Drive:", dst)
else:
    print("No out_path found. Run infer() first or check previous cells.")


No out_path found. Run infer() first or check previous cells.
